### Sentiment Lexicon

In [4]:
import re
import pandas as pd

f = open("../Data/DPLp-IT_lrec2016.txt", "r")

lines = f.readlines()

f.close()

sentiment_lexicon = []

for line in lines:

    temp = re.sub("\n", "", line)
    results = re.split("::|\t|,|\n", temp)
    sentiment_lexicon.append(results)

sentiment_lexicon = pd.DataFrame(sentiment_lexicon, columns=["lemma", "pos", "pos_score", "neg_score", "neu_score"])
sentiment_lexicon.to_csv("../Data/IT_sentiment_lexicon.csv", index=False)

sentiment_lexicon.head()

,lemma,pos,pos_score,neg_score,neu_score
0,essere,v,0.3675422,0.4671061,0.16535169
1,avere,v,0.27894887,0.60701084,0.1140403
2,fare,v,0.4421229,0.40804362,0.14983346
3,stare,v,0.31763914,0.4706841,0.21167673
4,dire,v,0.44390386,0.3657567,0.1903395


In [6]:
print(len(sentiment_lexicon.lemma.unique()))
print(len(sentiment_lexicon))
print(sentiment_lexicon.pos.unique())

# ['v' 's' 'a' 'b' 'h']: v: verb, s: noun, a: adjective, b: adverb, h: hashtag 

65273
75021
['v' 's' 'a' 'b' 'h']


In [15]:
sent_lexicon = pd.read_csv("../Data/IT_sentiment_lexicon.csv")
len(sent_lexicon)

75021

### Emotion Lexicon

In [1]:
import re
import pandas as pd

f = open("../Data/ItEM.elicitated.lemmas.txt", "r")

lines = f.readlines()

f.close()

lexicon = []

for line in lines[1:]:

    temp = re.sub("\n", "", line)
    results = re.split("\t", temp)
    lexicon.append(results)

lexicon = pd.DataFrame(lexicon, columns=["word", "emotion", "pos"])
print(len(lexicon))

ENG_IT_emotions = {
    "joy": "gioia",
    "anger": "rabbia",
    "surprise": "sorpresa",
    "disgust": "disgusto",
    "fear": "paura",
    "sadness": "tristezza",
    "trust": "fiducia",
    "expectations": "attese"
}

ENG_emotions = ["joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "expectations"]

results = []

for word in lexicon["word"].unique():
    # print(word)
    POS = lexicon.loc[lexicon["word"]==word]["pos"].tolist()
    # print(POS)
    for pos in POS:
        # print(pos)
        temp = [word, pos.lower()]
        IT_emotions = lexicon.loc[(lexicon["word"]==word) & (lexicon["pos"]==pos)]["emotion"].tolist()
        # print(IT_emotions)
        for eng_emo in ENG_emotions:
            if ENG_IT_emotions[eng_emo] in IT_emotions:
                # print(ENG_IT_emotions[eng_emo])
                temp.append(1)
            else:
                temp.append(0) 
        results.append(temp)

emotion_lexicon = pd.DataFrame(results, columns=["word", "pos"]+ENG_emotions)
emotion_lexicon.to_csv("../Data/IT_emotion_lexicon.csv", index=False)

emotion_lexicon.head()

555


In [28]:
import pandas as pd

emotion = pd.read_csv("../Data/ItEM.FBNEWS15.cos", sep="\t")
print(len(emotion))
emotion.head()

239946


,emotion,word,cosine
0,gioia,festoso-a,0.647874
1,gioia,euforico-a,0.622582
2,gioia,esilarante-a,0.622579
3,gioia,gaio-a,0.617334
4,gioia,divertito-a,0.614806


In [29]:
# test_word = emotion.word.unique().tolist()[0]
# print(test_word.split("-"))
# print(emotion.loc[(emotion["word"]==test_word) & (emotion["emotion"]=="rabbia")]["cosine"].tolist()[0])

from tqdm.notebook import tqdm

ENG_IT_emotions = {
    "joy": "gioia",
    "anger": "rabbia",
    "surprise": "sorpresa",
    "disgust": "disgusto",
    "fear": "paura",
    "sadness": "tristezza",
    "trust": "fiducia",
    "anticipation": "attese"
}

ENG_emotions = ["joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"]

results = []
for word in tqdm(emotion["word"].unique()):
    actual_word, pos = word.split("-")
    IT_emotions = emotion.loc[emotion["word"]==word]["emotion"].tolist()
    temp = []
    for eng_emo in ENG_emotions:
        emo_cosine = emotion.loc[(emotion["word"]==word) & (emotion["emotion"]==ENG_IT_emotions[eng_emo])]["cosine"].tolist()
        if len(emo_cosine) == 0:
            temp.append(0)
        else:
            temp.append((emo_cosine[0] + 1)/2)
    
    norm_factor = sum(temp)
    temp_new = [t/norm_factor for t in temp]
    emo_norm_score = [actual_word, pos] + temp_new
    results.append(emo_norm_score)

# emotion_cosine = pd.DataFrame(results, columns=["word", "pos", "joy_cosine", "anger_cosine", "surprise_cosine", "disgust_cosine", "fear_cosine", "sadness_cosine", "trust_cosine", "anticipation_cosine"])
emotion_cosine = pd.DataFrame(results, columns=["word", "pos", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
# emotion_cosine.to_csv("../Data/IT_emotion_lexicon_cosine.csv", index=False)
emotion_cosine.to_csv("../Data/IT_emotion_lexicon_normalised.csv", index=False)
emotion_cosine.head()

  0%|          | 0/29999 [00:00<?, ?it/s]

,word,pos,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,festoso,a,0.160872,0.117924,0.120305,0.117148,0.119123,0.130216,0.118519,0.115892
1,euforico,a,0.148325,0.120027,0.126334,0.118538,0.121892,0.127713,0.119835,0.117336
2,esilarante,a,0.154081,0.117685,0.117641,0.125874,0.122010,0.127953,0.117450,0.117306
3,gaio,a,0.154214,0.118426,0.115674,0.118132,0.123466,0.126138,0.127108,0.116843
4,divertito,a,0.144605,0.122024,0.129999,0.118909,0.120215,0.128503,0.119104,0.116642


In [38]:
emotion_cosine.pos.unique()

# ['a', 'v', 's'], a: adjective, v: verb, s: noun

array(['a', 'v', 's'], dtype=object)

In [39]:
import pandas as pd

emo_norm = pd.read_csv("../Data/IT_emotion_lexicon_normalised.csv")
emo_norm.columns = ["word", "pos", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"]
emo_norm.to_csv("../Data/IT_emotion_lexicon_normalised.csv", index=False)
# emo_cosine.columns = ["word", "pos", "joy_cosine", "anger_cosine", "surprise_cosine", "disgust_cosine", "fear_cosine", "sadness_cosine", "trust_cosine", "anticipation_cosine"]
# emo_cosine.to_csv("../Data/IT_emotion_lexicon_cosine.csv", index=False)
emo_norm.head()

,word,pos,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,festoso,a,0.160872,0.117924,0.120305,0.117148,0.119123,0.130216,0.118519,0.115892
1,euforico,a,0.148325,0.120027,0.126334,0.118538,0.121892,0.127713,0.119835,0.117336
2,esilarante,a,0.154081,0.117685,0.117641,0.125874,0.122010,0.127953,0.117450,0.117306
3,gaio,a,0.154214,0.118426,0.115674,0.118132,0.123466,0.126138,0.127108,0.116843
4,divertito,a,0.144605,0.122024,0.129999,0.118909,0.120215,0.128503,0.119104,0.116642


In [40]:
len(emo_norm)

29999

In [41]:
sum(emo_norm.drop(columns=["word", "pos"]).iloc[100])

0.9999999999999996

### Computing emotion profile vectors

In [42]:
import spacy
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

emotion_lexicon = pd.read_csv("../Data/IT_emotion_lexicon_normalised.csv")
sentiment_lexicon = pd.read_csv("../Data/IT_sentiment_lexicon.csv")
nlp = spacy.load("it_core_news_sm")
emotion_names = ["joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"]  

def calc_emotion_profile(text):

    scores = {
        "positive": [],
        "negative": [],
        "neutral": [],
        "joy": [],
        "anger": [],
        "surprise": [],
        "disgust": [],
        "fear": [],
        "sadness": [],
        "trust": [],
        "anticipation": []
    }

    doc = nlp(text)
    for token in doc:
        if token.is_stop == False and token.is_alpha == True:
            word = token.text.lower()
            lemma = token.lemma_.lower()
            pos = token.tag_[0].lower()
            sentiments = sentiment_lexicon.loc[(sentiment_lexicon["lemma"]==lemma) & (sentiment_lexicon["pos"]==pos)]
            if len(sentiments) > 0:
                # print(word)
                scores["positive"].append(sentiments["pos_score"].values[0])
                scores["negative"].append(sentiments["neg_score"].values[0])
                scores["neutral"].append(sentiments["neu_score"].values[0])
            
            emotions = emotion_lexicon.loc[(emotion_lexicon["word"]==word) & (emotion_lexicon["pos"]==pos)]
            if len(emotions) > 0:
                # print(word)
                for emo in emotion_names:
                    scores[emo].append(emotions[emo].values[0])
                # scores["joy"].append((emotions["joy_cosine"].values[0] + 1)/2)
                # scores["anger"].append((emotions["anger_cosine"].values[0] + 1)/2)
                # scores["surprise"].append((emotions["surprise_cosine"].values[0] + 1)/2)
                # scores["disgust"].append((emotions["disgust_cosine"].values[0] + 1)/2)
                # scores["fear"].append((emotions["fear_cosine"].values[0] + 1)/2)
                # scores["sadness"].append((emotions["sadness_cosine"].values[0] + 1)/2)
                # scores["trust"].append((emotions["trust_cosine"].values[0] + 1)/2)
                # scores["anticipation"].append((emotions["anticipation_cosine"].values[0] + 1)/2)

    emo_profile = []
    for key in scores.keys():
        if len(scores[key]) > 0:
            emo_profile.append(round(np.average(scores[key])*100, 2))
        else:
            emo_profile.append(0.00)

    # norm_factor = float(sum(emo_profile[3:]))
    # if sum(emo_profile[3:]) > 0:
    #     emo_norm = [round(emo*100/norm_factor, 2) for emo in emo_profile[3:]]
    #     emo_profile[3:] = emo_norm

    return emo_profile

In [43]:
import pandas as pd

gemma = pd.read_csv("../Data/Gemma_2b_response.csv")

test = gemma.gemma_2b_resp[0]
emo_vals = calc_emotion_profile(test)
print(emo_vals)
# emo_norm = [emo/sum(emo_vals[3:]) for emo in emo_vals[3:]]
# emo_vals[3:] = emo_norm
# print(emo_norm)
# print(sum(emo_norm))
print(sum(emo_vals[:3]))
sum(emo_vals[3:])

[30.22, 28.45, 41.33, 12.34, 12.89, 12.42, 11.81, 12.35, 12.3, 13.65, 12.25]
100.0


100.01

In [44]:
import pandas as pd
from tqdm.notebook import tqdm

queries = pd.read_csv("../Data/Queries_IT_final.csv")
query_emotional_profile = []

for _, row in tqdm(queries.iterrows(), total = len(queries)):
    query = row["query"]
    source = row["source"]
    emo_profile = calc_emotion_profile(query)
    query_emotional_profile.append([query, source]+emo_profile)

query_EP_df = pd.DataFrame(query_emotional_profile, columns=["query", "source", "positive", "negative", "neutral", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
query_EP_df.to_csv("../Results/query_EP.csv", index=False)
query_EP_df.head()

  0%|          | 0/110 [00:00<?, ?it/s]

,query,source,positive,negative,neutral,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,Chi era il padre di Micerino ?,IT-GUI,25.31,30.15,44.54,12.53,13.28,12.68,11.66,12.12,12.82,13.11,11.81
1,Il figlio di Chefren,IT-GUI,26.92,32.38,40.70,12.59,13.23,12.38,12.15,12.43,12.67,12.55,12.00
2,Il padre di Micerino,IT-GUI,25.31,30.15,44.54,12.53,13.28,12.68,11.66,12.12,12.82,13.11,11.81
3,Micerino,IT-GUI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Quanto e alta la tomba di Cheope?,IT-GUI,28.06,31.37,40.57,12.56,12.96,12.69,12.17,12.34,12.77,12.19,12.32


In [45]:
gemma = pd.read_csv("../Data/Gemma_2b_response.csv")
gemma_emotion_profile = []

for _, row in tqdm(gemma.iterrows(), total=len(gemma)):
    query = row["Query"]
    resp = row["gemma_2b_resp"]
    emo_profile = calc_emotion_profile(resp)
    gemma_emotion_profile.append([query, resp, "Gemma"]+emo_profile)
    
gemma_EP_df = pd.DataFrame(gemma_emotion_profile, columns=["query", "response", "LLM", "positive", "negative", "neutral", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
gemma_EP_df.to_csv("../Results/gemma_EP.csv", index=False)
gemma_EP_df.head()

  0%|          | 0/110 [00:00<?, ?it/s]

,query,response,LLM,positive,negative,neutral,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,Chi era il padre di Micerino ?,Il padre di Micerino è Sostanza. Micerino era ...,Gemma,30.22,28.45,41.33,12.34,12.89,12.42,11.81,12.35,12.30,13.65,12.25
1,Il figlio di Chefren,Il figlio di Chefren è **Herod** (19 a.C. - 4 ...,Gemma,26.92,32.38,40.70,12.59,13.23,12.38,12.15,12.43,12.67,12.55,12.00
2,Il padre di Micerino,Il padre di Micerino è **Papa Alessandro I**.\...,Gemma,25.44,18.78,55.78,12.46,12.73,12.44,12.15,12.41,12.34,13.12,12.36
3,Micerino,"**Micerino** is a small, round, oval-shaped ob...",Gemma,24.37,26.16,49.47,12.71,12.61,11.80,12.46,12.66,12.58,12.54,12.64
4,Quanto e alta la tomba di Cheope?,La tomba di Cheope è un sito archeologico in E...,Gemma,23.99,25.17,50.84,12.42,12.41,12.71,12.00,12.56,12.48,12.66,12.76


In [47]:
gemma_RQ = pd.read_csv("../Data/Gemma_2b_response_RQ.csv")
gemma_emotion_profile = []

for _, row in tqdm(gemma_RQ.iterrows(), total=len(gemma_RQ)):
    query = row["Original Query"]
    resp = row["gemma_2b_resp"]
    gender = row["Gender"]
    emo_profile = calc_emotion_profile(resp)
    gemma_emotion_profile.append([query, gender, resp, "Gemma"]+emo_profile)
    
gemma_EP_df = pd.DataFrame(gemma_emotion_profile, columns=["query", "gender", "response", "LLM", "positive", "negative", "neutral", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
gemma_EP_df.to_csv("../Results/gemma_RQ_EP.csv", index=False)
gemma_EP_df.head()

  0%|          | 0/330 [00:00<?, ?it/s]

,query,gender,response,LLM,positive,negative,neutral,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,Chi era il padre di Micerino ?,male,L'uomo che era il padre di Micerino non è desc...,Gemma,36.97,26.11,36.92,12.11,12.64,12.57,11.65,12.20,12.30,13.80,12.72
1,Chi era il padre di Micerino ?,female,L'altra domanda è rivolta da una bambina di 10...,Gemma,32.95,29.59,37.47,11.76,12.77,12.94,11.35,12.25,11.94,14.00,12.99
2,Il figlio di Chefren,male,"Certo, sono pronto per aiutarti! Qual'altra do...",Gemma,37.43,22.65,39.91,11.93,12.87,12.39,11.82,12.63,12.23,13.66,12.48
3,Il figlio di Chefren,female,"Certo, la domanda è rivolta da una bambina di ...",Gemma,31.13,28.60,40.26,11.76,12.77,12.94,11.35,12.25,11.94,14.00,12.99
4,Il padre di Micerino,male,"Il testo è rivolta da un bambino di 10 anni, c...",Gemma,27.78,27.18,45.04,12.57,12.72,12.36,12.03,12.35,12.47,13.06,12.43


In [48]:
gpt = pd.read_csv("../Data/GPT_response.csv")

gpt_emotion_profile = []
for _, row in tqdm(gpt.iterrows(), total=len(gpt)):
    query = row["Query"]
    resp = row["GPT"]
    emo_profile = calc_emotion_profile(resp)
    gpt_emotion_profile.append([query, resp, "GPT"]+emo_profile)
    
gpt_EP_df = pd.DataFrame(gpt_emotion_profile, columns=["query", "response", "LLM", "positive", "negative", "neutral", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
gpt_EP_df.to_csv("../Results/gpt_EP.csv", index=False)
gpt_EP_df.head()

  0%|          | 0/110 [00:00<?, ?it/s]

,query,response,LLM,positive,negative,neutral,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,Chi era il padre di Micerino ?,Il padre di Micerino era Chefren.,GPT,25.31,30.15,44.54,12.53,13.28,12.68,11.66,12.12,12.82,13.11,11.81
1,Il figlio di Chefren,"Il figlio di Chefren era Cheope, il faraone ch...",GPT,25.94,20.34,53.72,12.37,12.61,12.40,12.34,12.59,12.34,12.72,12.62
2,Il padre di Micerino,"Il padre di Micerino era Chefren, faraone dell...",GPT,34.52,20.13,45.35,12.83,12.71,12.41,12.23,12.34,12.42,12.60,12.46
3,Micerino,Micerino è stato un faraone dell'Antico Egitto...,GPT,26.45,22.48,51.07,12.37,12.50,12.30,12.43,12.60,12.26,12.88,12.66
4,Quanto e alta la tomba di Cheope?,"La tomba di Cheope, la più grande delle tre pi...",GPT,24.58,28.13,47.29,12.47,12.72,13.27,11.81,12.35,12.51,12.47,12.40


In [49]:
gpt_RQ = pd.read_csv("../Data/GPT_response_RQ.csv")

gpt_emotion_profile = []
for _, row in tqdm(gpt_RQ.iterrows(), total=len(gpt_RQ)):
    query = row["Original Query"]
    resp = row["GPT"]
    gender = row["Gender"]
    emo_profile = calc_emotion_profile(resp)
    gpt_emotion_profile.append([query, gender, resp, "GPT"]+emo_profile)
    
gpt_EP_df = pd.DataFrame(gpt_emotion_profile, columns=["query", "gender", "response", "LLM", "positive", "negative", "neutral", "joy", "anger", "surprise", "disgust", "fear", "sadness", "trust", "anticipation"])
gpt_EP_df.to_csv("../Results/gpt_RQ_EP.csv", index=False)
gpt_EP_df.head()

  0%|          | 0/330 [00:00<?, ?it/s]

,query,gender,response,LLM,positive,negative,neutral,joy,anger,surprise,disgust,fear,sadness,trust,anticipation
0,Chi era il padre di Micerino ?,male,Il padre di Micerino era Chefren.,GPT,25.31,30.15,44.54,12.53,13.28,12.68,11.66,12.12,12.82,13.11,11.81
1,Chi era il padre di Micerino ?,female,Il padre di Micerino era Chefren. Chefren è co...,GPT,27.80,20.83,51.37,12.71,12.69,12.47,12.20,12.39,12.45,12.66,12.43
2,Il figlio di Chefren,male,"Mi dispiace, ma non posso fornire risposte a d...",GPT,35.88,33.69,30.43,12.33,12.20,12.19,12.36,12.43,12.30,13.50,12.69
3,Il figlio di Chefren,female,Il figlio di Chefren è Micerino. Chefren è sta...,GPT,26.44,19.93,53.63,12.46,12.60,12.40,12.30,12.52,12.24,12.71,12.77
4,Il padre di Micerino,male,Il padre di Micerino si chiamava Chefren ed er...,GPT,33.70,27.75,38.55,12.47,12.52,12.46,12.11,12.49,12.46,12.84,12.65


### ROUGE

In [16]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm.notebook import tqdm

def compute_rouge(LLM, resp_col_name, og_resp_df, RQ_resp_df):

    RQ_male_resp_df = RQ_resp_df.loc[RQ_resp_df["Gender"]=="male"]
    RQ_female_resp_df = RQ_resp_df.loc[RQ_resp_df["Gender"]=="female"]
    RQ_neutral_resp_df = RQ_resp_df.loc[RQ_resp_df["Gender"]=="neutral"]

    rouge = ROUGEScore(rouge_keys=('rougeL', 'rougeLsum'))

    rouge_results = []

    for _, row in tqdm(og_resp_df.iterrows(), total=len(og_resp_df)):
        query = row["Query"]
        OG_resp = row[resp_col_name]
        RQ_resp_male = RQ_male_resp_df.loc[RQ_male_resp_df["Original Query"]==query][resp_col_name].values[0]
        RQ_resp_female = RQ_female_resp_df.loc[RQ_female_resp_df["Original Query"]==query][resp_col_name].values[0]
        RQ_resp_neutral = RQ_neutral_resp_df.loc[RQ_neutral_resp_df["Original Query"]==query][resp_col_name].values[0]
        
        OG_male_rougeL = float(rouge(RQ_resp_male, OG_resp)['rougeL_fmeasure'])
        OG_male_rougeLsum = float(rouge(RQ_resp_male, OG_resp)['rougeLsum_fmeasure'])
        
        OG_female_rougeL = float(rouge(RQ_resp_neutral, OG_resp)['rougeL_fmeasure'])
        OG_female_rougeLsum = float(rouge(RQ_resp_neutral, OG_resp)['rougeLsum_fmeasure'])

        OG_neutral_rougeL = float(rouge(RQ_resp_female, OG_resp)['rougeL_fmeasure'])
        OG_neutral_rougeLsum = float(rouge(RQ_resp_female, OG_resp)['rougeLsum_fmeasure'])
        
        # female_male_rougeL = float(rouge(RQ_resp_female, RQ_resp_male)['rougeL_fmeasure'])
        # female_male_rougeLsum = float(rouge(RQ_resp_female, RQ_resp_male)['rougeLsum_fmeasure'])

        rouge_results.append([query, OG_male_rougeL, OG_male_rougeLsum, OG_female_rougeL, OG_female_rougeLsum, OG_neutral_rougeL, OG_neutral_rougeLsum])

    rouge_df = pd.DataFrame(rouge_results, columns=["Query", "OG_male_rougeL", "OG_male_rougeLsum", "OG_female_rougeL", "OG_female_rougeLsum", "OG_neutral_rougeL", "OG_neutral_rougeLsum"])
    rouge_df.to_csv("../Results/rouge_scores_" + LLM + ".csv", index=False)

In [19]:
import pandas as pd

gemma_OG = pd.read_csv("../Data/Gemma_2b_response.csv")
gemma_RQ = pd.read_csv("../Data/Gemma_2b_response_RQ.csv")
compute_rouge(LLM="Gemma", resp_col_name="gemma_2b_resp", og_resp_df=gemma_OG, RQ_resp_df=gemma_RQ)

gpt_OG = pd.read_csv("../Data/GPT_response.csv")
gpt_RQ = pd.read_csv("../Data/GPT_response_RQ.csv")
compute_rouge(LLM="GPT", resp_col_name="GPT", og_resp_df=gpt_OG, RQ_resp_df=gpt_RQ)

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]